In [70]:
# imports
import pickle
import pandas as pd
import numpy as np
import re
from fuzzywuzzy import fuzz
pd.set_option('display.max_rows', 500)

In [2]:
### Debugging test
Dict = {
("recipeid", 10): {
    "qty": 10, "unit": "kg", "name": "carrot cake", "other": "aad", "input": "10 kg of carrot cake", "comment": "adsa"
},
("recipeid2", 11): {
    "qty": 3, "unit": "mg", "name": "apple", "other": "aead", "input": "3mg apple", "comment": "ada"
},
("recipeid3", 12): {
    "qty": 22, "unit": "lb", "name": "water bottle", "other": "d", "input": "22 lb water bottle", "comment": "ada"
}
}
print(Dict)

{('recipeid', 10): {'qty': 10, 'unit': 'kg', 'name': 'carrot cake', 'other': 'aad', 'input': '10 kg of carrot cake', 'comment': 'adsa'}, ('recipeid2', 11): {'qty': 3, 'unit': 'mg', 'name': 'apple', 'other': 'aead', 'input': '3mg apple', 'comment': 'ada'}, ('recipeid3', 12): {'qty': 22, 'unit': 'lb', 'name': 'water bottle', 'other': 'd', 'input': '22 lb water bottle', 'comment': 'ada'}}


In [57]:
dict_units = [];
for key, value in Dict.items():
    dict_units.append(value['unit'])
dict_units

['kg', 'mg', 'lb']

In [58]:
dict_names = [];
for key, value in Dict.items():
    dict_names.append(value['name'])
dict_names

['carrot cake', 'apple', 'water bottle']

In [125]:
def dfreplace(df, colname, org, new):
    newcol = df[colname].copy()
    newcol = [re.sub(org, new, string) for string in newcol]
    df[colname] = newcol
    return df

def match(ing_in, ing_ser = None):
    if ing_in is None or isinstance(ing_in, float):
        return -1
    else:
        res = ing_ser.str.contains(ing_in, regex=False)
        if res.any():
            new = ing_ser[res]
            #print(new)
            res = new.index[0]
            # print(res)
            return res
        else:
            return -1
def matches(ing_in, ingredients):
    out = ing_in.apply(match, ing_ser=ingredients)
    matched = (out != -1).sum()
    return out
    print('Percent matched %.2f' % (matched / len(out)))

def join_ingredients(names, matches, usda):
    names['match'] = matches
    joined = pd.merge(names, usda[['desc']], how='left', left_on='match', right_index=True)
    return joined

In [6]:
# load inputs
ingredients = pd.read_excel('../data/raw/nutrients.xlsx')
ing_in = pickle.load(open('../data/raw/full_ingredients.pkl', 'rb'))
ingredients.head()

,NDB_No,Shrt_Desc,Water_(g),Energ_Kcal,Protein_(g),Lipid_Tot_(g),Ash_(g),Carbohydrt_(g),Fiber_TD_(g),Sugar_Tot_(g),...,Vit_K_(µg),FA_Sat_(g),FA_Mono_(g),FA_Poly_(g),Cholestrl_(mg),GmWt_1,GmWt_Desc1,GmWt_2,GmWt_Desc2,Refuse_Pct
0,1001,"BUTTER,WITH SALT",15.87,717,0.85,81.11,2.11,0.06,0.0,0.06,...,7.0,51.368,21.021,3.043,215.0,5.00,"1 pat, (1"" sq, 1/3"" high)",14.2,1 tbsp,0.0
1,1002,"BUTTER,WHIPPED,W/ SALT",16.72,718,0.49,78.30,1.62,2.87,0.0,0.06,...,4.6,45.390,19.874,3.331,225.0,3.80,"1 pat, (1"" sq, 1/3"" high)",9.4,1 tbsp,0.0
2,1003,"BUTTER OIL,ANHYDROUS",0.24,876,0.28,99.48,0.00,0.00,0.0,0.00,...,8.6,61.924,28.732,3.694,256.0,12.80,1 tbsp,205.0,1 cup,0.0
3,1004,"CHEESE,BLUE",42.41,353,21.40,28.74,5.11,2.34,0.0,0.50,...,2.4,18.669,7.778,0.800,75.0,28.35,1 oz,17.0,1 cubic inch,0.0
4,1005,"CHEESE,BRICK",41.11,371,23.24,29.68,3.18,2.79,0.0,0.51,...,2.5,18.764,8.598,0.784,94.0,132.00,"1 cup, diced",113.0,"1 cup, shredded",0.0


In [ ]:
# percentage of units that don't have matches, put the ones without matches into a separate file or output

In [7]:
ingr_unit1 = ingredients[["Shrt_Desc", "GmWt_Desc1"]]
ingr_unit2 = ingredients[["Shrt_Desc", "GmWt_Desc2"]]
ingr_unit1.columns =["ingredient", "AU"]
ingr_unit2.columns =["ingredient", "AU"]
ingr_unit = pd.concat([ingr_unit1, ingr_unit2])

ingr_unit['amount'], ingr_unit['unit'] = ingr_unit['AU'].str.split(' ', 1).str
ingr_unit['unit'], ingr_unit['misc'] = ingr_unit['unit'].str.split(',', 1).str

ingr_unit.head(10000)

,ingredient,AU,amount,unit,misc
0,"BUTTER,WITH SALT","1 pat, (1"" sq, 1/3"" high)",1,pat,"(1"" sq, 1/3"" high)"
1,"BUTTER,WHIPPED,W/ SALT","1 pat, (1"" sq, 1/3"" high)",1,pat,"(1"" sq, 1/3"" high)"
2,"BUTTER OIL,ANHYDROUS",1 tbsp,1,tbsp,NaN
3,"CHEESE,BLUE",1 oz,1,oz,NaN
4,"CHEESE,BRICK","1 cup, diced",1,cup,diced
5,"CHEESE,BRIE",1 oz,1,oz,NaN
6,"CHEESE,CAMEMBERT",1 oz,1,oz,NaN
7,"CHEESE,CARAWAY",1 oz,1,oz,NaN
8,"CHEESE,CHEDDAR","1 cup, diced",1,cup,diced
9,"CHEESE,CHESHIRE",1 oz,1,oz,NaN


In [9]:
# checking which columns exist
count = 0
allkeys = []
for key, val in ing_in.items():
    keys.append(key)
    
allkeys = set(allkeys)
print(allkeys)

{'comment', 'input', 'unit', 'name', 'display', 'other', 'qty'}


In [21]:
# populating column dictionary
cols = {}

for key in allkeys:
    cols[key] = []

In [22]:
# creating inputs for dataframe creation
keys = []
counter = 0
for key, val in ing_in.items():
    keys.append(key)
    for colkey in cols.keys():
        if colkey not in val:
            cols[colkey].append(np.NaN)
        else:
            cols[colkey].append(val[colkey])
    if counter % 100000 == 0:
        print('%d / %d ' % (counter, len(ing_in)))
    counter += 1

0 / 1277382 
100000 / 1277382 
200000 / 1277382 
300000 / 1277382 
400000 / 1277382 
500000 / 1277382 
600000 / 1277382 
700000 / 1277382 
800000 / 1277382 
900000 / 1277382 
1000000 / 1277382 
1100000 / 1277382 
1200000 / 1277382 


In [23]:
for key in allkeys:
    print(len(cols[key]))
test = []
test.append(1)

1277382
1277382
1277382
1277382
1277382
1277382
1277382


In [20]:
len(test)

1

In [24]:
# build ingredient dataframe
ingdf = pd.DataFrame(data=cols, index=pd.MultiIndex.from_tuples(keys, names=['rid', 'num']))

In [27]:
ingdf.sort_index(level=['rid', 'num'], inplace=True)
ingdf = ingdf.reindex(['name', 'qty', 'unit', 'comment', 'input', 'other', 'display'], axis=1)
ingdf.head()

name    qty        unit  \
rid                             num                                             
../AFIrLizRK7C5isREQ0hUdXoNXTpS 0    creamy salad dressing  2 1/2         cup   
                                1           yellow mustard    3/4         cup   
                                2              white sugar    1/4         cup   
                                3            white vinegar      2  tablespoon   
                                4                     eggs     12         NaN   

                                                                              comment  \
rid                             num                                                     
../AFIrLizRK7C5isREQ0hUdXoNXTpS 0                              such as Miracle Whip®)   
                                1                                            prepared   
                                2                                                 NaN   
                                3                                                 NaN   
                                4    hard-boiled, peeled and cut into 1/4-inch slices   

                                                                                 input  \
rid                             num                                                      
../AFIrLizRK7C5isREQ0hUdXoNXTpS 0    2 1/2 cups creamy salad dressing (such as Mira...   
                                1                      3/4 cup prepared yellow mustard   
                                2                                  1/4 cup white sugar   
                                3                          2 tablespoons white vinegar   
                                4    12 hard-boiled eggs, peeled and cut into 1/4-i...   

                                    other  \
rid                             num         
../AFIrLizRK7C5isREQ0hUdXoNXTpS 0       (   
                                1     NaN   
                                2     NaN   
                                3     NaN   
                                4     NaN   

                                                                               display  
rid                             num                                                     
../AFIrLizRK7C5isREQ0hUdXoNXTpS 0    <span class='qty'>2 1/2</span><span class='uni...  
                                1    <span class='qty'>3/4</span><span class='unit'...  
                                2    <span class='qty'>1/4</span><span class='unit'...  
                                3    <span class='qty'>2</span><span class='unit'>t...  
                                4    <span class='qty'>12</span><span class='commen...

In [37]:
print(len(ingdf))
names = ingdf['name'].str.lower()
print(ingredients.columns)
ingredients['desc'] = ingredients['Shrt_Desc'].str.lower()
ingredients.rename(columns={'NDB_No': 'id'}, inplace=True)
ingredients.set_index('id', inplace=True, drop=True)
ti = ingredients[['desc']]

1277382
Index(['NDB_No', 'Shrt_Desc', 'Water_(g)', 'Energ_Kcal', 'Protein_(g)',
       'Lipid_Tot_(g)', 'Ash_(g)', 'Carbohydrt_(g)', 'Fiber_TD_(g)',
       'Sugar_Tot_(g)', 'Calcium_(mg)', 'Iron_(mg)', 'Magnesium_(mg)',
       'Phosphorus_(mg)', 'Potassium_(mg)', 'Sodium_(mg)', 'Zinc_(mg)',
       'Copper_mg)', 'Manganese_(mg)', 'Selenium_(µg)', 'Vit_C_(mg)',
       'Thiamin_(mg)', 'Riboflavin_(mg)', 'Niacin_(mg)', 'Panto_Acid_mg)',
       'Vit_B6_(mg)', 'Folate_Tot_(µg)', 'Folic_Acid_(µg)', 'Food_Folate_(µg)',
       'Folate_DFE_(µg)', 'Choline_Tot_ (mg)', 'Vit_B12_(µg)', 'Vit_A_IU',
       'Vit_A_RAE', 'Retinol_(µg)', 'Alpha_Carot_(µg)', 'Beta_Carot_(µg)',
       'Beta_Crypt_(µg)', 'Lycopene_(µg)', 'Lut+Zea_ (µg)', 'Vit_E_(mg)',
       'Vit_D_µg', 'Vit_D_IU', 'Vit_K_(µg)', 'FA_Sat_(g)', 'FA_Mono_(g)',
       'FA_Poly_(g)', 'Cholestrl_(mg)', 'GmWt_1', 'GmWt_Desc1', 'GmWt_2',
       'GmWt_Desc2', 'Refuse_Pct', 'desc'],
      dtype='object')


In [45]:
ti['desc'].head()
count = ingredients.loc[:, 'desc'].copy().str.replace(',', ' ')
count = [len(re.findall("[a-zA-Z_]+", string)) for string in count]
ti['wcnt'] = count
ti['ccnt'] = ti['desc'].str.count(',')

C:\Users\bpiv4\Anaconda3\lib\site-packages\ipykernel_launcher.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  after removing the cwd from sys.path.
C:\Users\bpiv4\Anaconda3\lib\site-packages\ipykernel_launcher.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  """


,desc,wcnt,ccnt
id,,,
1001,"butter,with salt",3,1
1002,"butter,whipped,w/ salt",4,2
1003,"butter oil,anhydrous",3,1
1004,"cheese,blue",2,1
1005,"cheese,brick",2,1
1006,"cheese,brie",2,1
1007,"cheese,camembert",2,1
1008,"cheese,caraway",2,1
1009,"cheese,cheddar",2,1


In [83]:
targword = ['cheese', 'yogurt']
contains = pd.Series(False, index=ti.index)
for word in targword:
    contains = contains | ti['desc'].str.contains(word)

ser1 = ti['wcnt'] == 2
ser2 = ti['ccnt'] == 1
update =  (ti['wcnt'] == 2) & (ti['ccnt'] == 1) | contains
update_subset = ti.loc[update, 'desc']
regex = re.compile(r'(\w+),(\w+)')
updated = [re.sub(regex, r'\2 \1', string) for string in update_subset]
print(updated)
ti2 = ti.copy()
ti2.loc[update , 'desc'] = updated
ti2.head(2000)

['blue cheese', 'brick cheese', 'brie cheese', 'camembert cheese', 'caraway cheese', 'cheddar cheese', 'cheshire cheese', 'colby cheese', 'cottage cheese,lrg crmd or sml curd', 'cottage cheese,w crmd/fruit', 'cottage cheese,uncrmd nonfat,lrg dry or sml curd', 'cottage cheese,2 lowfat% milkfat', 'cottage cheese,1 lowfat% milkfat', 'cream cheese', 'edam cheese', 'feta cheese', 'fontina cheese', 'gjetost cheese', 'gouda cheese', 'gruyere cheese', 'limburger cheese', 'monterey cheese', 'mozzarella cheese,whl milk', 'mozzarella cheese,whl lo milk moist', 'mozzarella cheese,part skim milk', 'mozzarella cheese,lo part moist-skim', 'muenster cheese', 'neufchatel cheese', 'parmesan cheese,grated', 'parmesan cheese,hard', 'port cheese de salut', 'provolone cheese', 'ricotta cheese,whole milk', 'ricotta cheese,part skim milk', 'romano cheese', 'roquefort cheese', 'swiss cheese', 'tilsit cheese', 'past cheese american process,fort w/ vitamin d', 'past cheese pimento process', 'past cheese swiss pr

,desc,wcnt,ccnt
id,,,
1001,"butter,with salt",3,1
1002,"butter,whipped,w/ salt",4,2
1003,"butter oil,anhydrous",3,1
1004,blue cheese,2,1
1005,brick cheese,2,1
1006,brie cheese,2,1
1007,camembert cheese,2,1
1008,caraway cheese,2,1
1009,cheddar cheese,2,1


In [112]:
# word replacements
ti2 = dfreplace(ti2, 'desc', 'w/ ', 'with ')
ti2 = dfreplace(ti2, 'desc', 'w/', 'with ')
ti2 = dfreplace(ti2, 'desc', 'crm ', 'cream ')
ti2 = dfreplace(ti2, 'desc', 'crm,', 'cream,')
ti2 = dfreplace(ti2, 'desc', 'crm$', 'cream')
ti2 = dfreplace(ti2, 'desc', 'frz,', 'frozen,')
ti2 = dfreplace(ti2, 'desc', 'frz ', 'frozen ')
ti2 = dfreplace(ti2, 'desc', 'wo/ ', 'without ')
ti2 = dfreplace(ti2, 'desc', 'wo/', 'without ')
ti2 = dfreplace(ti2, 'desc', 'free fat', 'fat free')
ti2 = dfreplace(ti2, 'desc', 'free fat', 'fat free')
ti2 = dfreplace(ti2, 'desc', 'fat free', 'fat-free')
ti2 = dfreplace(ti2, 'desc', 'lo ', 'low ')
ti2 = dfreplace(ti2, 'desc', 'lofat', 'low-fat')
ti2 = dfreplace(ti2, 'desc', 'low fat', 'low-fat')
ti2 = dfreplace(ti2, 'desc', 'pln ', 'plain ')
ti2 = dfreplace(ti2, 'desc', 'pln,', 'plain,')
ti2 = dfreplace(ti2, 'desc', 'pln$', 'plain')
ti2 = dfreplace(ti2, 'desc', 'choc,', 'chocolate,')
ti2 = dfreplace(ti2, 'desc', 'choc ', 'chocolate ')
ti2 = dfreplace(ti2, 'desc', 'choc$', 'chocolate')
ti2 = dfreplace(ti2, 'desc', 'sndwch ', 'sandwich ')
ti2 = dfreplace(ti2, 'desc', 'sndwch,', 'sandwich,')
ti2 = dfreplace(ti2, 'desc', 'sndwch$', 'sandwich')
ti2 = dfreplace(ti2, 'desc', 'whl,', 'whole,')
ti2 = dfreplace(ti2, 'desc', 'whl ', 'whole ')
ti2 = dfreplace(ti2, 'desc', 'whl$', 'whole')
ti2 = dfreplace(ti2, 'desc', 'frsh,', 'fresh,')
ti2 = dfreplace(ti2, 'desc', 'frsh ', 'fresh ')
ti2 = dfreplace(ti2, 'desc', 'frsh$', 'fresh')
ti2 = dfreplace(ti2, 'desc', 'cereals', 'cereal')
ti2 = dfreplace(ti2, 'desc', 'frstd,', 'frosted,')
ti2 = dfreplace(ti2, 'desc', 'frstd ', 'frosted ')
ti2 = dfreplace(ti2, 'desc', 'frstd$', 'frosted')
ti2.head(2000)

,desc,wcnt,ccnt
id,,,
1001,"butter,with salt",3,1
1002,"butter,whipped,with salt",4,2
1003,"butter oil,anhydrous",3,1
1004,blue cheese,2,1
1005,brick cheese,2,1
1006,brie cheese,2,1
1007,camembert cheese,2,1
1008,caraway cheese,2,1
1009,cheddar cheese,2,1


In [109]:
names = pd.DataFrame(index=names.index, data={'name': names.values})

Exception: Data must be 1-dimensional

In [110]:
names.head(2000)

name
rid                             num                                                   
../AFIrLizRK7C5isREQ0hUdXoNXTpS 0                                creamy salad dressing
                                1                                       yellow mustard
                                2                                          white sugar
                                3                                        white vinegar
                                4                                                 eggs
..1fLeH8MEEaHhoimrhhh39zHkJiywe 0                                          clementines
                                1                                            red onion
                                2                                             cilantro
                                3                                            olive oil
                                4                                   white wine vinegar
                                5                   ancho chile powder or chile powder
                                6                                       cayenne pepper
                                7                     2 3/4-inch-thick new york steaks
..1q2exyLrX7e57eayExiARxQNYn0he 0                                            olive oil
                                1                                           beef chuck
                                2                                salt and black pepper
                                3                                               onions
                                4                                               garlic
                                5                                             red wine
                                6                                                 beef
                                7                                        wheat berries
                                8                                             bay leaf
                                9                                                thyme
                                10                                            parsnips
                                11                                             parsley
..2GZ19dHw2ysetJ8mQ1A6vRmvD.RD6 0                                            olive oil
                                1                                                onion
                                2                                               garlic
                                3                                                  egg
                                4                                                 milk
                                5                                                 pork
                                6                            small-diced russet potato
                                7                                          breadcrumbs
                                8                                               nutmeg
                                9                                                thyme
                                10                  salt and fresh ground black pepper
                                11                                         puff pastry
                                12                                               flour
..2dhKN7RleNgC3OLFwapkerklssbOC 0                                                bread
                                1                                      unsalted butter
                                2                                                  oil
                                3                                         white onions
                                4                                               celery
                                5                                               apples
                                6                                               

In [114]:
# sort ingredients by length
ti2['length'] = ti2['desc'].str.len()
ti2.sort_values(by='length', ascending=True, inplace=True)

In [ ]:
subset_names = names.head(2000)
my_matches = matches(subset_names['name'], ti2['desc'])

In [126]:
join_ingredients(subset_names, my_matches, ti2)

C:\Users\bpiv4\Anaconda3\lib\site-packages\ipykernel_launcher.py:27: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy


name  \
rid                             num                                                      
../AFIrLizRK7C5isREQ0hUdXoNXTpS 0                                creamy salad dressing   
                                1                                       yellow mustard   
                                2                                          white sugar   
                                3                                        white vinegar   
                                4                                                 eggs   
..1fLeH8MEEaHhoimrhhh39zHkJiywe 0                                          clementines   
                                1                                            red onion   
                                2                                             cilantro   
                                3                                            olive oil   
                                4                                   white wine vinegar   
                                5                   ancho chile powder or chile powder   
                                6                                       cayenne pepper   
                                7                     2 3/4-inch-thick new york steaks   
..1q2exyLrX7e57eayExiARxQNYn0he 0                                            olive oil   
                                1                                           beef chuck   
                                2                                salt and black pepper   
                                3                                               onions   
                                4                                               garlic   
                                5                                             red wine   
                                6                                                 beef   
                                7                                        wheat berries   
                                8                                             bay leaf   
                                9                                                thyme   
                                10                                            parsnips   
                                11                                             parsley   
..2GZ19dHw2ysetJ8mQ1A6vRmvD.RD6 0                                            olive oil   
                                1                                                onion   
                                2                                               garlic   
                                3                                                  egg   
                                4                                                 milk   
                                5                                                 pork   
                                6                            small-diced russet potato   
                                7                                          breadcrumbs   
                                8                                               nutmeg   
                                9                                                thyme   
                                10                  salt and fresh ground black pepper   
                                11                                         puff pastry   
                                12                                               flour   
..2dhKN7RleNgC3OLFwapkerklssbOC 0                                                bread   
                                1                                      unsalted butter   
                                2                                                  oil   
                                3                                         white onions   
                                4                                               celery   
                                